### Tunning dos modelos

Até aqui construímos os modelos usando apenas os valores padrão dos parâmetros de cada modelo (script modelos_básicos). Nesse script irei mudar a abordagem utilizada, trazendo mais complexidade para as análises com o objetivo de tentar aperfeiçoar a modelagem e obter melhores resultados. Para isso, irei trabalhar com o GridSearch, um método que permite testar os modelos com diferentes valores de parâmetros e obter aquele com a melhor performance. No caso, vamos vou assumir a métrica acurácia como a melhor indicadora de desempenho para o meu problema. Vamos lá:


___

In [40]:
import pandas as pd
import numpy as np
import pickle as pickle
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

Primeiro passo: concatenar as bases que estão salvas separadamente em treino e teste:

In [41]:
with open("credit.pkl", "rb") as f:
    X_credit_train, y_credit_train, X_credit_test, y_credit_test = pickle.load(f)

In [42]:
X_credit = np.concatenate((X_credit_train, X_credit_test), axis=0)
y_credit = np.concatenate((y_credit_train,y_credit_test),axis=0)
X_credit.shape, y_credit.shape

((2000, 3), (2000,))

### Tunning dos modelos

Aplicando o GridSearch para encontrar os melhores parâmetros dos algoritmos (Árvore de Decisão, Random Forest, KNN e Regressão Logística). Não apliquei esta lógica ao algoritmo Naive-Bayes porque ele não possui parâmetros importantes a serem modificados.

In [43]:
# Função para retornar melhor modelo:
def returnBestParams(model, params):
    grid_search = GridSearchCV(estimator=model,param_grid=params)
    grid_search.fit(X_credit,y_credit)
    best_params = grid_search.best_params_
    best_accuracy = grid_search.best_score_
    return {"Best params": best_params, "Best accuracy": best_accuracy}

### Árvore de Decisão

In [46]:
tree_decision = DecisionTreeClassifier()
tree_params = {'criterion': ['gini', 'entropy'],
              'splitter': ['best', 'random'],
              'min_samples_split': [2, 5, 10],
              'min_samples_leaf': [1, 5, 10]}

In [47]:
returnBestParams(tree_decision, tree_params)

{'Best params': {'criterion': 'gini',
  'min_samples_leaf': 1,
  'min_samples_split': 2,
  'splitter': 'best'},
 'Best accuracy': 0.9855}

### Random Forest

In [50]:
random_forest = RandomForestClassifier()
random_forest_params = {'criterion': ['gini', 'entropy'],
              'n_estimators': [10, 40, 100, 150],
              'min_samples_split': [2, 5, 10],
              'min_samples_leaf': [1, 5, 10]}

In [51]:
returnBestParams(random_forest, random_forest_params)

{'Best params': {'criterion': 'gini',
  'min_samples_leaf': 1,
  'min_samples_split': 5,
  'n_estimators': 100},
 'Best accuracy': 0.986}

### KNN

In [52]:
knn = KNeighborsClassifier()
knn_params = {'n_neighbors': [3, 5, 10, 20],
              'p': [1, 2]}

In [53]:
returnBestParams(knn, knn_params)

{'Best params': {'n_neighbors': 5, 'p': 1}, 'Best accuracy': 0.982}

### Regressão Logística

In [54]:
log_reg = LogisticRegression()
log_reg_params = {'tol': [0.0001, 0.00001, 0.000001],
              'C': [1.0, 1.5, 2.0],
              'solver': ['lbfgs', 'sag', 'saga']}

In [55]:
returnBestParams(log_reg,log_reg_params)

{'Best params': {'C': 1.0, 'solver': 'lbfgs', 'tol': 0.0001},
 'Best accuracy': 0.9484999999999999}

### Próximos Passos

No próximo script (validação cruzada), irei realizar realizar uma forma de validação e análise dos modelos que nos ajudará a ter mais confiança sobre qual deles tem um melhor desempenho, baseado em testes estatísticos: a validação cruzada.